# Turkey Sound Binary Classifier
Available at: https://www.kaggle.com/c/dont-call-me-turkey/

In [62]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.applications import MobileNetV2
import skimage.transform
%matplotlib inline

In [63]:
df = pd.read_json("train.json")
df.head(3)

,audio_embedding,end_time_seconds_youtube_clip,is_turkey,start_time_seconds_youtube_clip,vid_id
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",70,0,60,kDCk3hLIVXo
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",40,1,30,DPcGzqHoo7Y
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",240,1,230,7yM63MTHh5k


In [91]:
def load_dataset(path):
    df = pd.read_json(path)
    y = df.is_turkey.values
    X = embedder(df)
    return X,y

def embedder(df):
    embeddings = []
    for embedding in df['audio_embedding']:
        embedding = np.asarray(embedding)
        l = embedding.shape[0]
        not_ten_sec = (l != 10)
        if not_ten_sec:
            embedding = np.pad(embedding,((0,10-l),(0,0)),mode = 'constant')  
        embeddings.append(embedding)
    return np.asarray(embeddings)

X,y = load_dataset('train.json')
val = pd.read_json('test.json')
X_val = embedder(val)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [94]:
#Create Keras Model
model = Sequential()
model.add(BatchNormalization(input_shape=(10, 128)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(128,recurrent_dropout=0.4,activation='relu')))
model.add(Dense(1,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_21 (Batc (None, 10, 128)           512       
_________________________________________________________________
dropout_21 (Dropout)         (None, 10, 128)           0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 256)               263168    
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 257       
Total params: 263,937
Trainable params: 263,681
Non-trainable params: 256
_________________________________________________________________


In [95]:
early_stop = EarlyStopping(monitor='val_loss', 
                           min_delta=0.00001, 
                           patience=5, 
                           mode='min', 
                           verbose=1)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=100, 
          batch_size = 256,
          validation_data=(X_test, y_test),
          callbacks=[early_stop]
         )

Train on 1075 samples, validate on 120 samples
Epoch 1/100
1075/1075 [==============================] - 11s 10ms/step - loss: 0.5152 - acc: 0.7460 - val_loss: 0.2963 - val_acc: 0.8750
Epoch 2/100
1075/1075 [==============================] - 1s 763us/step - loss: 0.1901 - acc: 0.9256 - val_loss: 0.2291 - val_acc: 0.8917
Epoch 3/100
1075/1075 [==============================] - 1s 756us/step - loss: 0.1498 - acc: 0.9460 - val_loss: 0.2231 - val_acc: 0.9250
Epoch 4/100
1075/1075 [==============================] - 1s 1ms/step - loss: 0.1277 - acc: 0.9479 - val_loss: 0.2284 - val_acc: 0.9000
Epoch 5/100
1075/1075 [==============================] - 1s 912us/step - loss: 0.1187 - acc: 0.9553 - val_loss: 0.2235 - val_acc: 0.9083
Epoch 6/100
1075/1075 [==============================] - 1s 865us/step - loss: 0.1052 - acc: 0.9619 - val_loss: 0.2216 - val_acc: 0.9083
Epoch 7/100
1075/1075 [==============================] - 1s 929us/step - loss: 0.0888 - acc: 0.9619 - val_loss: 0.2299 - val_acc: 0.9

In [96]:
preds = model.predict_classes(X_val)
val['is_turkey'] = preds
val.head()

,audio_embedding,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip,vid_id,is_turkey
0,"[[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...",10,0,pyKh38FXD3E,0
1,"[[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...",40,30,THhP1idrWXA,0
2,"[[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...",40,30,jsw3T6GY2Nw,0
3,"[[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...",24,14,nFkXTMHcjMU,0
4,"[[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...",40,30,Au8g9kAlrLQ,1


In [97]:
sub = val[['vid_id','is_turkey']].copy()
sub.set_index('vid_id',inplace = True)

In [98]:
sub.to_csv('TurkeySub.csv')